In [1]:
import openai
import os

from dotenv import find_dotenv, load_dotenv


_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

MODEL_NAME = os.environ["MODEL_NAME"]
MODEL_PROVIDER = os.environ["MODEL_PROVIDER"]

In [2]:
import nest_asyncio

from langsmith import traceable
from openai import OpenAI
from typing import Dict, List
from utils import get_vector_db_retriever


RAG_SYSTEM_PROMPT = """
	You are an assistant for question-answering tasks. 
	Use the following pieces of retrieved context to answer the latest question in the conversation. 
	If you don't know the answer, just say that you don't know. 
	Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()


@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)
                         

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)


@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[Dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )


@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

USER_AGENT environment variable not set, consider setting it to identify your requests.
Fetching pages: 100%|##########| 239/239 [06:14<00:00,  1.57s/it]


In [6]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "course-dataset"

def is_concise_enough(reference_outputs: Dict, outputs: Dict) -> Dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: Dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix=MODEL_NAME
)

/home/rizka-amalia/.pyenv/versions/3.13.0/envs/langchain-venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'gpt-4o-mini-95a5547d' at:
https://smith.langchain.com/o/ed964922-bae8-547e-a7ee-294ce654f5d2/datasets/02aef164-b3d3-4b94-abd2-8353d7a69ae5/compare?selectedSessions=870520db-1c86-41fb-8eeb-f2e4626d7f44




12it [00:37,  3.09s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I integrate LangSmith with third-party...,To integrate LangSmith with third-party applic...,None,To integrate LangSmith with third-party applic...,1,4.988269,1227adcb-7b6c-4938-8305-4b46a768ed38,2babc2de-cf16-4620-809c-4b324fabc8eb
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith using LangChain...,1,2.756209,809068d2-c980-4b3c-a5af-2bc35b809851,542f0982-5589-4eb6-bb15-69bab09eac75
2,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,4.059666,3b95e1fd-d873-416a-a1e0-4dfd5567e595,2e78e2c1-b71a-4824-a02d-0e5cd184dcb0
3,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.326069,59bd7b3d-a71c-44af-bd24-8e87bd867a9a,2275d99e-01d9-4ee9-ad7a-37ff616dd38e
4,Can LangSmith be used for finetuning and model...,The context does not specify whether LangSmith...,None,"Yes, LangSmith can be used for fine-tuning and...",1,3.018236,e110ac0b-0579-4295-827a-b3b505f325e1,e58397bd-96ee-444e-ad4b-e14bbb252681
5,Does LangSmith support offline evaluation?,I don't know.,None,"Yes, LangSmith supports offline evaluation thr...",1,1.611300,536070df-7079-4b5c-8967-0af9fa2a2878,90160bd3-92c5-4bd0-ba8d-57a1c4f2b1db
6,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,2.103757,8855502c-fce8-47e5-a444-5a6310e9759f,e00f6375-bbf9-4214-8b84-568b5f54ab2b
7,What testing capabilities does LangSmith have?,LangSmith provides several testing capabilitie...,None,LangSmith offers capabilities for creating dat...,1,3.791636,8400f6da-2a46-413c-bb64-b6d278756bf0,dbe66d3c-adcf-45cd-86b0-87fa7154721b
8,What is LangSmith used for in three sentences?,"LangSmith is a platform for developing, monito...",None,LangSmith is a platform designed for the devel...,1,3.853847,8b2b72b4-f400-4311-ad30-46d4e68a4c9d,70f51183-4c92-414c-a851-081916c6d900
9,How do I pass metadata in with @traceable?,To pass metadata with the @traceable decorator...,None,You can pass metadata with the @traceable deco...,1,2.474349,e1a57cb2-4472-4aa2-a4b1-29c5b50b8924,c39ba924-2356-4747-8868-d306f6c5d941


In [7]:
from langsmith import evaluate


def target_function(inputs: Dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-e7a6ef60' at:
https://smith.langchain.com/o/ed964922-bae8-547e-a7ee-294ce654f5d2/datasets/02aef164-b3d3-4b94-abd2-8353d7a69ae5/compare?selectedSessions=5d442286-889e-4f07-8058-7088abcb64dd




12it [00:44,  3.74s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I integrate LangSmith with third-party...,To integrate LangSmith with third-party applic...,None,To integrate LangSmith with third-party applic...,1,9.121544,1227adcb-7b6c-4938-8305-4b46a768ed38,00a15543-cfd7-46cd-9c11-411e3c87ff37
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith using LangChain...,1,3.508257,809068d2-c980-4b3c-a5af-2bc35b809851,04a6400d-5741-4ecb-9296-2a893021504b
2,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.233883,3b95e1fd-d873-416a-a1e0-4dfd5567e595,c3f150b1-d89d-441d-b332-83f292b9d809
3,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents,...",None,"Yes, LangSmith can be used to evaluate agents....",1,1.839476,59bd7b3d-a71c-44af-bd24-8e87bd867a9a,2340b198-e2bd-464a-b777-b389b0c0bc60
4,Can LangSmith be used for finetuning and model...,I don't know.,None,"Yes, LangSmith can be used for fine-tuning and...",1,3.431526,e110ac0b-0579-4295-827a-b3b505f325e1,aec95544-e7f4-4647-a5aa-2b358dcbb9d4
5,Does LangSmith support offline evaluation?,I don't know.,None,"Yes, LangSmith supports offline evaluation thr...",1,1.283132,536070df-7079-4b5c-8967-0af9fa2a2878,f56abec6-d2c4-4d98-bc1b-64a46bc3aa6d
6,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. The...",None,"Yes, LangSmith supports online evaluation as a...",1,3.014494,8855502c-fce8-47e5-a444-5a6310e9759f,2cc09cc7-659f-48f1-b18c-de6e6adff0c5
7,What testing capabilities does LangSmith have?,LangSmith allows developers to create datasets...,None,LangSmith offers capabilities for creating dat...,1,2.253226,8400f6da-2a46-413c-bb64-b6d278756bf0,555ab208-4da9-4e24-b992-494548f02ae2
8,What is LangSmith used for in three sentences?,LangSmith is a platform designed for the devel...,None,LangSmith is a platform designed for the devel...,1,2.532079,8b2b72b4-f400-4311-ad30-46d4e68a4c9d,89e20d94-c274-44d6-a961-33b01fdcaf11
9,How do I pass metadata in with @traceable?,To pass metadata with the @traceable decorator...,None,You can pass metadata with the @traceable deco...,1,3.042210,e1a57cb2-4472-4aa2-a4b1-29c5b50b8924,3ddd2ed9-62d8-4e1a-9c61-33a298b45ac6
